In [ ]:
# If you like, upvote beezus666's script and the ensemeble sources
#https://www.kaggle.com/code/beezus666/ensemble-weighted-average

In [ ]:
from catboost import CatBoostClassifier
import gc; gc.enable()
from sklearn import *
import pandas as pd
import numpy as np
import numba, os

sub1 = pd.read_csv('../input/ensemble-weighted-average/submission.csv')
traini = pd.read_csv('/kaggle/input/amex-default-prediction/train_data.csv', parse_dates=['S_2'], chunksize=400_000, iterator=True)
testi = pd.read_csv('/kaggle/input/amex-default-prediction/test_data.csv', parse_dates=['S_2'], chunksize=400_000, iterator=True) 
labels = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')

In [ ]:
%%time
train = []
for df in traini:
    if len(train)>0: train = pd.concat([train, df])
    else: train = df[:]
    train.sort_values(by=['S_2'], inplace=True)
    train.reset_index(drop=True, inplace=True)
    train.drop_duplicates(subset=['customer_ID'], keep='last', inplace=True)
    del df; gc.collect()
train = pd.merge(train, labels, how='inner', on=['customer_ID'])
del labels; gc.collect()
col = [c for c in train if c not in ['customer_ID', 'target','S_2']]
train.fillna(0).to_csv('train.csv', index=False)
del train; del traini; gc.collect()

test = []
for df in testi:
    if len(test)>0: test = pd.concat([test, df])
    else: test = df[:]
    test.sort_values(by=['S_2'], inplace=True)
    test.reset_index(drop=True, inplace=True)
    test.drop_duplicates(subset=['customer_ID'], keep='last', inplace=True)
    del df; gc.collect()
test.fillna(0).to_csv('test.csv', index=False)
del test; del testi; gc.collect()

In [ ]:
# https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# https://www.kaggle.com/code/rohanrao/amex-competition-metric-implementations
class CB_Amex_Metric(object):
    def get_final_error(self, error, weight):
        return error
    def is_max_optimal(self):
        return True
    def evaluate(self, y_pred, y_true, weight):
        y_pred = y_pred[0]
        indices = np.argsort(y_pred)[::-1]
        preds, target = y_pred[indices], y_true[indices]
        weight = 20.0 - target * 19.0
        cum_norm_weight = (weight / weight.sum()).cumsum()
        four_pct_mask = cum_norm_weight <= 0.04
        d = np.sum(target[four_pct_mask]) / np.sum(target)
        weighted_target = target * weight
        lorentz = (weighted_target / weighted_target.sum()).cumsum()
        gini = ((lorentz - cum_norm_weight) * weight).sum()
        n_pos = np.sum(target)
        n_neg = target.shape[0] - n_pos
        gini_max = 10 * n_neg * (n_pos + 20 * n_neg - 19) / (n_pos + 20 * n_neg)
        g = gini / gini_max
        return 0.5 * (g + d), 0

In [ ]:
train = pd.read_csv('train.csv')
cat_features = ['B_30', 'B_31', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
for c in cat_features: train[c] = train[c].astype(str)

x1, x2, y1, y2 = model_selection.train_test_split(train[col], train.target, test_size=0.20, random_state=22)
del train; gc.collect()
os.remove ('train.csv')

clf = CatBoostClassifier(iterations=1000, random_state=22, nan_mode='Min', eval_metric=CB_Amex_Metric())
clf.fit(x1, y1, eval_set=[(x2, y2)], cat_features=cat_features,  verbose=50)
preds = clf.predict_proba(x2)[:, 1]

In [ ]:
test = pd.read_csv('test.csv')
for c in cat_features: test[c] = test[c].astype(str)
test['prediction'] = clf.predict_proba(test[col])[:, 1]
sub2 = test[['customer_ID', 'prediction']]
del test;  gc.collect()
os.remove ('test.csv')

In [ ]:
sub2.columns = ['customer_ID', 'prediction2']
blend = pd.merge(sub1, sub2, how='inner', on='customer_ID')
blend.prediction = (blend.prediction * 0.955 + blend.prediction2 * 0.045)
blend[['customer_ID', 'prediction']].to_csv('submission.csv', index=False)